## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pasvalys,LT,56.06,24.40,67.32,96,64,5.82,broken clouds
1,1,Atuona,PF,-9.80,-139.03,77.83,71,1,17.92,clear sky
2,2,Ribeira Grande,PT,38.52,-28.70,72.01,85,55,8.61,broken clouds
3,3,Talnakh,RU,69.49,88.40,61.65,86,96,4.45,light rain
4,4,New Norfolk,AU,-42.78,147.06,55.92,63,53,5.01,light rain


In [ ]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
temp_min=float(input("Enter Min Temp"))
temp_max=float(input("Enter Max Temp"))

In [ ]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc meth
Chosen_Temps_df=city_data_df.loc[(city_data_df["Max Temp"]>=temp_min)&(city_data_df["Max Temp"]>=temp_max)]

In [ ]:
#4a. Determine if there are any empty rows
Chosen_Temps_df.isnull().sum()

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=Chosen_Temps_df.dropna()

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i,row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    latitude=row["Lat"]    
    longitude=row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url_base="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    result_hotels=requests.get(url_base,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    hotel_df.loc[i,"Hotel Name"]=result_hotels["results"][0]["name"]
        

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

In [ ]:
# 8a. Create the output File (CSV)
output_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
fig=gmaps.figure()
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig